In [186]:

import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt


failure_data = pd.read_csv(r'Failure_modelling\Failure_Data\failure_data.csv', index_col=0)
pipe_data = pd.read_csv(r'Failure_modelling\Failure_Data\pipe_data.csv', index_col=0)

In [187]:
ac_pipes = pipe_data[pipe_data.Materiaalcode == 'MAT_AC']
ac_failures = failure_data[failure_data.storing_materiaal == 'MAT_AC']
model_failures = ac_failures[['storing_materiaal', 'storing_diameter', 'storing_melddatum', 'storing_aanlegjaar', 'age_at_failure']].copy()
model_failures.loc[:,'age_at_failure'] = model_failures.loc[:,'age_at_failure'].astype(int)
model_pipes = ac_pipes[['Aanlegjaar', 'Lengte', 'Diameter', 'Materiaalcode']].copy()
model_pipes = model_pipes[model_pipes.Aanlegjaar > 1]
print(model_pipes.head())
print(model_failures.head())


         Aanlegjaar      Lengte  Diameter Materiaalcode
2166783        1974  368.595185       200        MAT_AC
2110184        1990    0.500100       150        MAT_AC
2144900        1989   35.832798       200        MAT_AC
2104664        1979   15.248820       100        MAT_AC
2182521        1973  280.118530       100        MAT_AC
   storing_materiaal  storing_diameter    storing_melddatum  \
4             MAT_AC              50.0  2010-03-10 00:00:00   
7             MAT_AC             160.0  2010-11-19 00:00:00   
8             MAT_AC              50.0  2010-04-16 00:00:00   
9             MAT_AC             100.0  2010-11-15 00:00:00   
10            MAT_AC             100.0  2010-07-19 00:00:00   

   storing_aanlegjaar  age_at_failure  
4          1960-01-01            50.0  
7          1963-01-01            47.0  
8          1969-01-01            41.0  
9          1955-01-01            55.0  
10         1968-01-01            42.0  


In [188]:
diameter_groups = [[0, 160],
                   [160, 300],
                   [300, 1000],
                   [1000, 10000]]

construction_year_groups = []
for i in range(1920, 2000 + 1, 10):
    construction_year_groups.append([i, i+10])

model_pipes['Diameter_group'] = None
model_failures['Diameter_group'] = None
model_pipes['Construction_year_group'] = None
model_failures['Construction_year_group'] = None
model_pipes['Group_number'] = None
model_failures['Group_number'] = None
model_pipes['Group_id'] = None
model_failures['Group_id'] = None

group_number = 1
for diameter_group in diameter_groups:
    diameter_mask_failures = (model_failures.storing_diameter >= diameter_group[0]) & (model_failures.storing_diameter < diameter_group[1])
    model_failures.loc[diameter_mask_failures, 'Diameter_group'] = f'{diameter_group[0]} - {diameter_group[1]}'
    
    diameter_mask_pipes = (model_pipes.Diameter >= diameter_group[0]) & (model_pipes.Diameter < diameter_group[1])
    model_pipes.loc[diameter_mask_pipes, 'Diameter_group'] = f'{diameter_group[0]} - {diameter_group[1]}'

    for construction_year_group in construction_year_groups:
        construction_year_mask_failures = (pd.to_datetime(model_failures.storing_aanlegjaar, format='%Y-%m-%d').dt.year>= construction_year_group[0]) & (pd.to_datetime(model_failures.storing_aanlegjaar, format='%Y-%m-%d').dt.year < construction_year_group[1])
        model_failures.loc[construction_year_mask_failures, 'Construction_year_group'] = f'{construction_year_group[0]} - {construction_year_group[1]}'
        model_failures.loc[(diameter_mask_failures & construction_year_mask_failures), 'Group_number'] = group_number
        
        model_failures.loc[(diameter_mask_failures & construction_year_mask_failures), 'Group_id'] = int(f'{diameter_group[0]}{diameter_group[1]}{construction_year_group[0]}{construction_year_group[1]}')
        
        construction_year_mask_pipes = (pd.to_datetime(model_pipes.Aanlegjaar, format='%Y').dt.year >= construction_year_group[0]) & (pd.to_datetime(model_pipes.Aanlegjaar, format='%Y').dt.year < construction_year_group[1])
        model_pipes.loc[construction_year_mask_pipes, 'Construction_year_group'] = f'{construction_year_group[0]} - {construction_year_group[1]}'
        model_pipes.loc[(diameter_mask_pipes & construction_year_mask_pipes), 'Group_number'] = group_number
        
        model_pipes.loc[(diameter_mask_pipes & construction_year_mask_pipes), 'Group_id'] = int(f'{diameter_group[0]}{diameter_group[1]}{construction_year_group[0]}{construction_year_group[1]}')

        group_number += 1

In [189]:
model_pipes = model_pipes[model_pipes.Group_id.isin(model_failures.Group_id.unique())].copy()

In [190]:
model_failures = model_failures[model_failures.Group_id.isin(model_pipes.Group_id.unique())].copy()

In [191]:
model_failures.Group_id.unique()

array([16019601970, 16030019601970, 16019501960, 16019902000, 16019801990,
       16019401950, 16030019401950, 16030019801990, 16019701980,
       16030019701980, 300100019601970, 300100019701980, 16019301940,
       16030019501960, 16030019301940, 300100019501960, 16030019902000,
       300100019801990, None], dtype=object)

In [200]:
model_pipes.Group_number.unique()

TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [198]:
len(model_failures.Group_number.unique())

19